In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone -b feat/hadi-ibra/idiom-extension https://github.com/hadi-ibra/SICK_Summarization.git

Cloning into 'SICK_Summarization'...
remote: Enumerating objects: 218, done.
remote: Counting objects: 100% (138/138), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 218 (delta 92), reused 99 (delta 63), pack-reused 80
Receiving objects: 100% (218/218), 1.22 MiB | 6.26 MiB/s, done.
Resolving deltas: 100% (114/114), done.


In [12]:
!pip install -U sentence-transformers

  Using cached sentence_transformers-2.6.1-py3-none-any.whl.metadata (11 kB)
  Using cached transformers-4.39.3-py3-none-any.whl.metadata (134 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.3 MB/s eta 0:00:00
  Using cached huggingface_hub-0.22.2-py3-none-any.whl.metadata (12 kB)
  Using cached filelock-3.13.3-py3-none-any.whl.metadata (2.8 kB)
  Using cached fsspec-2024.3.1-py3-none-any.whl.metadata (6.8 kB)
  Using cached typing_extensions-4.11.0-py3-none-any.whl.metadata (3.0 kB)
  Using cached sympy-1.12-py3-none-any.whl.metadata (12 kB)
  Using cached Jinja2-3.1.3-py3-none-any.whl.metadata (3.3 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
 

In [13]:
!pip install tqdm

In [18]:
import requests
import bs4
import pandas as pd

def extract_all_tables_from_url(url):
    """Extracts all tables from the given URL and returns them concatenated as a single Pandas DataFrame.

    Args:
        url (str): The URL of the web page containing the tables.

    Returns:
        pandas.DataFrame: The concatenated tables as a DataFrame.
        None: If no tables were found on the page or an error occurred.
    """

    try:
        response = requests.get(url)

        # Retrieve the webpage and store it as an bs4.BeautifulSoup object
        # response from request call is parsed using html parser
        html_soup = bs4.BeautifulSoup(response.text, 'html.parser')

        all_tables = html_soup.find_all('table')

        if all_tables:
            dfs = []  # List to store DataFrames of each table
            for table in all_tables:
                header_row = []
                data_rows = []

                # Extract header
                if table.find('thead'):
                    for th in table.find('thead').find_all('th'):
                        header_row.append(th.text.strip())
                else:
                    header_row = [th.text.strip() for th in table.find('tr').find_all('th')]

                # Extract data rows
                for tr in table.find('tbody').find_all('tr'):
                    row = []
                    for td in tr.find_all('td'):
                        row.append(td.text.strip())
                    if len(row) < len(header_row):
                        row += [''] * (len(header_row) - len(row))
                    data_rows.append(row)

                dfs.append(pd.DataFrame(data_rows, columns=header_row))

            return pd.concat(dfs, ignore_index=True)  # Concatenate DataFrames
        else:
            return None

    except requests.exceptions.RequestException as e:
        print(f"An error occurred during the request: {e}")
        return None

# Example usage
url = "https://www.ef.com/wwen/english-resources/english-idioms/"
all_tables_df = extract_all_tables_from_url(url)

if all_tables_df is not None:
    print(all_tables_df)
else:
    print("No tables found on the page or an error occurred.")


                                                 Idiom  \
0                               A blessing in disguise   
1                                       A dime a dozen   
2                                 Beat around the bush   
3                               Better late than never   
4                                      Bite the bullet   
..                                                 ...   
150                            Well begun is half done   
151                             When it rains it pours   
152  You can catch more flies with honey than you c...   
153  You can lead a horse to water, but you can't m...   
154  You can't make an omelet without breaking some...   

                                               Meaning                  Usage  
0                a good thing that seemed bad at first  as part of a sentence  
1                                     Something common  as part of a sentence  
2    Avoid saying what you mean, usually because it...  as part

## Second Source for Idioms

In [ ]:
import requests, bs4
import pandas as pd

In [ ]:
def get_response(url):
    # Prepare GET request
    # calls url and get the response
    response = requests.get(url)

    # Retrieve the webpage and store it as an bs4.BeautifulSoup object
    # response from request call is parsed using html parser
    html_soup = bs4.BeautifulSoup(response.text, 'html.parser')
    return html_soup

In [ ]:
def get_meaning_from_url(url,tag,class_):

    html_soup = get_response('https://www.phrases.org.uk/idioms/' + url)

    quote_explanation = html_soup.find_all(tag, class_ = class_)
    if len(quote_explanation) >= 1:
        quote_explanation = str(quote_explanation[0].text)
    else:
        quote_explanation = ""

    return quote_explanation

In [ ]:
#when load page in your browser, you will notice important and correct written tags
#load all idioms in  tag a

#english idioms url https://www.phrases.org.uk/idioms/a-z/full-list.html
soup=get_response('https://www.phrases.org.uk/idioms/a-z/full-list.html')
a_idioms=soup.find_all('a')

In [ ]:
idioms=[]
meanings=[]
for e in a_idioms:
    if e.get('title'):
        idioms.extend([e.get('title')])
        #need to replace '../' in each href to nothing to connect right url
        meanings.extend([get_meaning_from_url(e.get('href').replace('../',''),"div","idiom-meaning")])

In [ ]:
#saving idioms and their meanings in dataframe
df=pd.DataFrame({"idioms":idioms,"meaning":meanings})


## Third Source for Idioms

In [ ]:
folder_path = '/content/drive/MyDrive/SICK_Summarization/data/Idioms/idioms.csv'  # Replace with your actual folder path
final_df = pd.read_csv(folder_path)

In [ ]:
merged_df = pd.concat([all_tables_df, df, final_df], ignore_index=True)

# Efficiently remove duplicates based on 'column1'
def remove_duplicates(df):
  return df.loc[~df['idioms'].duplicated()]

# Apply the function to remove duplicates
df_without_duplicates = remove_duplicates(merged_df.copy())

In [ ]:
df_without_duplicates.to_csv("/content/drive/MyDrive/SICK_Summarization/data/Idioms/all_idioms.csv")

## Initaliasing Idiom detection model and sentence embedding model

In [16]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
import torch
from sentence_transformers import SentenceTransformer,util


# Model for idiom detection
model = AutoModelForTokenClassification.from_pretrained("imranraad/idiom-xlm-roberta").to('cuda')
# tokenizer for idiom detection
tokenizer = AutoTokenizer.from_pretrained("imranraad/idiom-xlm-roberta")
# pipeline for idiom detection
pipeline_idioms = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

# Sentence embedding model
# embedding_model1 = SentenceTransformer('sentence-transformers/all-mpnet-base-v2').to('cuda')
embedding_model2 = SentenceTransformer('mixedbread-ai/mxbai-embed-large-v1').to('cuda')

/usr/local/lib/python3.10/site-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [17]:
# function for accounting for pronouns after detecting idioms using the pipeline
def identify_idioms(sentence):
    idioms = pipeline_idioms(sentence)
    identified_idioms = []
    current_idiom = ""

    for i, idiom in enumerate(idioms):
        if idiom['entity_group'] == '1':
            current_idiom += " " + idiom['word']  # Add space for better separation
        elif idiom['word'].lower() in ['a', 'the', 'an', 'her', 'his', 'its','her','him','own','their','our','you','my']:
            if i < len(idioms) - 1 and idioms[i + 1]['entity_group'] == '1':
              current_idiom += " " + idiom['word']  # Join the article/pronoun
        else:
            if current_idiom:
                identified_idioms.append(current_idiom.strip())
                current_idiom = ""
    # Add any idiom at the end
    if current_idiom:
        identified_idioms.append(current_idiom.strip())

    # Filter out single-word idioms
    filtered_idioms = [idiom for idiom in identified_idioms if len(idiom.split()) > 1]

    return filtered_idioms




In [19]:
# Load idiom mappings dataset
idiom_mappings = pd.read_csv("/content/drive/MyDrive/SICK_Summarization/data/Idioms/all_idioms.csv")  # Replace with your file path
idiom_mappings

,idioms,meaning
0,A blessing in disguise,a good thing that seemed bad at first
1,A dime a dozen,Something common
2,Beat around the bush,"Avoid saying what you mean, usually because it..."
3,Better late than never,Better to arrive late than not to come at all
4,Bite the bullet,To get something over with because it is inevi...
...,...,...
1029,School of hard knocks,"Learning through difficult experiences, as opp..."
1030,School of thought,Particular philosophy or way of thinking about...
1031,Schoolboy error,Very basic or foolish mistake
1032,Seeing eye to eye,Two or more people agree on something


### Compute the embeddings for all idioms

In [ ]:
precomputed_embeddings = {}
for index, row in idiom_mappings.iterrows():
  idiom = row["idioms"]
  meaning = row["meaning"]
  print(row)
  precomputed_embeddings[idiom] = embedding_model2.encode(idiom)

In [ ]:
import pickle


# Open a file in write binary mode ("wb")
with open("precomputed_embeddings.pkl", "wb") as f:
  # Use pickle.dump() to serialize the dictionary and write it to the file
  pickle.dump(precomputed_embeddings, f)


In [ ]:
# convert embeddings from numpy array to torch tensors

precomputed_embeddings_tensor = {}
for idiom, embedding in precomputed_embeddings.items():
    precomputed_embeddings_tensor[idiom] = torch.from_numpy(embedding)

precomputed_embeddings_tensor = torch.stack([e for e in precomputed_embeddings_tensor.values()]).to('cuda')


In [ ]:
# move file to drive!
import shutil
dest_folder = "/content/drive/MyDrive/SICK_Summarization/data/Idioms/"
source_folder = f"/content/precomputed_embeddings.pkl"
shutil.copy(source_folder, dest_folder)

In [14]:
import pickle
with open("/content/drive/MyDrive/SICK_Summarization/data/Idioms/precomputed_embeddings.pkl", "rb") as f:
  # Read the pickled data
  precomputed_embeddings = pickle.load(f)


In [21]:
import pandas as pd
import torch
from tqdm import tqdm
import torch.nn.functional as F

def annotate_conversations(data):
    """
    Annotates conversations in a dataset with idiom meanings using similarity search.

    Args:
        data: A dictionary representing the conversation data.
        precomputed_embeddings: A dictionary of precomputed idiom embeddings.

    Returns:
        The annotated data.
    """

    for conversation_id, conversation in tqdm(data.items()):
        # Collect sentences for batch processing
        sentences = [turn["sentence"] for turn in conversation.values()]
        identified_idioms_batches = [identify_idioms(sentence) for sentence in sentences]

        # Batch encode all sentences
        all_embeddings = embedding_model2.encode(sentences)

        # Process each turn and its corresponding embeddings
        for turn, sentence_embeddings, idioms_batch in zip(conversation.values(), all_embeddings, identified_idioms_batches):
            if idioms_batch:
                idiom_embeddings = torch.from_numpy(embedding_model2.encode(idioms_batch)).to('cuda')
                similarities = F.cosine_similarity(idiom_embeddings.unsqueeze(1), precomputed_embeddings_tensor.unsqueeze(0), dim=2)
                best_similarities, best_match_indices = similarities.max(dim=1)

                idiom_meanings = []
                for index, (idiom, idx) in enumerate(zip(idioms_batch, best_match_indices)):
                    best_match_idiom = list(precomputed_embeddings.keys())[idx.item()]
                    if best_similarities[index].item() > 0.8:
                        meaning = idiom_mappings.loc[idiom_mappings['idioms'] == best_match_idiom, 'meaning'].iloc[0]

                        annotated_sentence = turn["sentence"].replace(idiom, f"{idiom} ({meaning})")
                        idiom_meanings.append(f"{idiom} means {meaning}")

                        turn["sentence"] = annotated_sentence
                        turn["<|xidiom_meaning|>"] = idiom_meanings

    return data


In [ ]:


file_names = ["test", "train", "valid"]

for file_name in file_names:
  import json
  with open(f"/content/drive/MyDrive/SICK_Summarization/data/COMET_data/paracomet/dialogue/samsum/dialog_{file_name}_split5_collated.json") as f:
    data = json.load(f)
  annotated_data = annotate_conversations(data.copy())
  data_file = f'preprocessed_{file_name}_data.json'
  with open(data_file, 'w') as outfile:
        json.dump(annotated_data, outfile, indent=4)

  src = f"/content/preprocessed_{file_name}_data.json"
  dst = "/content/drive/MyDrive/SICK_Summarization/data/COMET_data/paracomet/dialogue/samsum/idioms"

  shutil.copy(src, dst)

